In [3]:
from  bs4 import BeautifulSoup
import pandas as pd
import requests


In [43]:
urls = ['https://www.basketball-reference.com/players/j/jamesle01/gamelog/2004/','https://www.basketball-reference.com/players/j/jamesle01/gamelog/2005/'
       ,'https://www.basketball-reference.com/players/j/jamesle01/gamelog/2006/','https://www.basketball-reference.com/players/j/jamesle01/gamelog/2007/'
       , 'https://www.basketball-reference.com/players/j/jamesle01/gamelog/2008/','https://www.basketball-reference.com/players/j/jamesle01/gamelog/2009/',
       'https://www.basketball-reference.com/players/j/jamesle01/gamelog/2010/','https://www.basketball-reference.com/players/j/jamesle01/gamelog/2011/',
       'https://www.basketball-reference.com/players/j/jamesle01/gamelog/2012/','https://www.basketball-reference.com/players/j/jamesle01/gamelog/2013/',
       'https://www.basketball-reference.com/players/j/jamesle01/gamelog/2014/','https://www.basketball-reference.com/players/j/jamesle01/gamelog/2015/',
       'https://www.basketball-reference.com/players/j/jamesle01/gamelog/2016/','https://www.basketball-reference.com/players/j/jamesle01/gamelog/2017/',
       'https://www.basketball-reference.com/players/j/jamesle01/gamelog/2018/','https://www.basketball-reference.com/players/j/jamesle01/gamelog/2019/']

tables = soup.find_all('table',id ='pgl_basic')

headers = tables[0].find('thead')

tableHeaders = []

for th in headers.find_all('th'):
    if th.text.strip() == 'Rk':
        continue
    tableHeaders.append(th.text.strip())
tableHeaders

['G',
 'Date',
 'Age',
 'Tm',
 '',
 'Opp',
 '',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'GmSc',
 '+/-']

In [36]:
def createLebronFrame (url):
    results = requests.get(url)

    soup = BeautifulSoup(results.content,'html.parser')


    tables = soup.find_all('table',id ='pgl_basic')
    

    rows = tables[0].find('tbody').find_all('tr')

    gameStats = []

    for row in rows:
        stats = []
        for game in row.find_all('td'):
            stats.append(game.text.strip())
        gameStats.append(stats)

    cleanStats = []

    for indx,game in enumerate(gameStats):
        if len(game) == 0 or len(game)< 11:
            continue;
        cleanStats.append(game)

    return cleanStats
    

    

In [44]:
gameStat2003_2013 = []
for url in urls:
    gameStat2003_2013.append(createLebronFrame(url))
    

len(gameStat2003_2013)

    

16

In [100]:
dfs = []
for season in gameStat2003_2013:
    dfs.append(pd.DataFrame(season,columns = tableHeaders))


Lebron_Df = pd.concat(dfs)

Lebron_Df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1198 entries, 0 to 54
Data columns (total 29 columns):
G       1198 non-null object
Date    1198 non-null object
Age     1198 non-null object
Tm      1198 non-null object
        1198 non-null object
Opp     1198 non-null object
        1198 non-null object
GS      1198 non-null object
MP      1198 non-null object
FG      1198 non-null object
FGA     1198 non-null object
FG%     1198 non-null object
3P      1198 non-null object
3PA     1198 non-null object
3P%     1198 non-null object
FT      1198 non-null object
FTA     1198 non-null object
FT%     1198 non-null object
ORB     1198 non-null object
DRB     1198 non-null object
TRB     1198 non-null object
AST     1198 non-null object
STL     1198 non-null object
BLK     1198 non-null object
TOV     1198 non-null object
PF      1198 non-null object
PTS     1198 non-null object
GmSc    1198 non-null object
+/-     1198 non-null object
dtypes: object(29)
memory usage: 280.8+ KB


In [101]:
Lebron_Df.columns.values[4] = 'Location'
Lebron_Df.columns.values[6] = 'Margin'

In [102]:
Lebron_Df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1198 entries, 0 to 54
Data columns (total 29 columns):
G           1198 non-null object
Date        1198 non-null object
Age         1198 non-null object
Tm          1198 non-null object
Location    1198 non-null object
Opp         1198 non-null object
Margin      1198 non-null object
GS          1198 non-null object
MP          1198 non-null object
FG          1198 non-null object
FGA         1198 non-null object
FG%         1198 non-null object
3P          1198 non-null object
3PA         1198 non-null object
3P%         1198 non-null object
FT          1198 non-null object
FTA         1198 non-null object
FT%         1198 non-null object
ORB         1198 non-null object
DRB         1198 non-null object
TRB         1198 non-null object
AST         1198 non-null object
STL         1198 non-null object
BLK         1198 non-null object
TOV         1198 non-null object
PF          1198 non-null object
PTS         1198 non-null object
GmSc

In [103]:

#formating the dataframe

Lebron_Df['Location']=Lebron_Df['Location'].apply(lambda row: 'H' if row == '' else '@')
Lebron_Df['Age']=Lebron_Df['Age'].apply(lambda row: row.split('-')[0])
Lebron_Df['Date']=pd.to_datetime(Lebron_Df['Date'], format="%Y-%m-%d")
Lebron_Df['+/-'] = Lebron_Df['+/-'].str.replace("+","")
Lebron_Df['FG%'] = Lebron_Df['FG%'].str.replace("%","")
Lebron_Df['3P%'] = Lebron_Df['3P%'].str.replace("%","")
Lebron_Df['FT%'] =Lebron_Df['FT%'].str.replace("%","")

#Lebron_Df['MP']=Lebron_Df['MP'].apply(lambda row: row.split(':')[0])

Lebron_Df= Lebron_Df.astype({'G': 'int32','FG': 'int32','MP':'int32','GmSc':'float64','+/-':'int32','MP':'int32',
 'FGA': 'int32',
 'FG%':'float64',
 '3P': 'int32',
 '3PA': 'int32',
 '3P%':'float64',
 'FT': 'int32',
 'FTA': 'int32',
 'FT%':'float64',
 'ORB': 'int32',
 'DRB': 'int32',
 'TRB': 'int32',
 'AST': 'int32',
 'STL': 'int32',
 'BLK': 'int32',
 'TOV': 'int32',
 'PF': 'int32',
 'PTS': 'int32'},errors='ignore')

In [104]:
Lebron_Df.dtypes

G                    int32
Date        datetime64[ns]
Age                 object
Tm                  object
Location            object
Opp                 object
Margin              object
GS                  object
MP                  object
FG                   int32
FGA                  int32
FG%                float64
3P                   int32
3PA                  int32
3P%                 object
FT                   int32
FTA                  int32
FT%                 object
ORB                  int32
DRB                  int32
TRB                  int32
AST                  int32
STL                  int32
BLK                  int32
TOV                  int32
PF                   int32
PTS                  int32
GmSc               float64
+/-                  int32
dtype: object

In [94]:

Lebron_Df.to_csv('Lebron Stats')

G                   object
Date        datetime64[ns]
Age                 object
Tm                  object
Location            object
Opp                 object
Margin              object
GS                  object
MP                  object
FG                  object
FGA                 object
FG%                 object
3P                  object
3PA                 object
3P%                 object
FT                  object
FTA                 object
FT%                 object
ORB                 object
DRB                 object
TRB                 object
AST                 object
STL                 object
BLK                 object
TOV                 object
PF                  object
PTS                 object
GmSc                object
+/-                 object
dtype: object

G                   object
Date        datetime64[ns]
Age                 object
Tm                  object
Location            object
Opp                 object
Margin              object
GS                  object
MP                  object
FG                  object
FGA                 object
FG%                 object
3P                  object
3PA                 object
3P%                 object
FT                  object
FTA                 object
FT%                 object
ORB                 object
DRB                 object
TRB                 object
AST                 object
STL                 object
BLK                 object
TOV                 object
PF                  object
PTS                 object
GmSc                object
+/-                 object
dtype: object